In [1]:
%pip install scikit-fuzzy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import numpy as np
import time as tm
import random as rdm

In [4]:
# define os domínios das entradas e saídas
fila_LO = ctrl.Antecedent(np.arange(0,101,1), 'fila_LO')
fila_NS = ctrl.Antecedent(np.arange(0,101,1), 'fila_NS')
tempo_semaforo = ctrl.Consequent(np.arange(30,120.5,0.5), 'tempo_semaforo')

In [5]:
# define as funções de pertinência
# fila_LO
fila_LO['baixo'] = fuzz.trapmf(fila_LO.universe, [0,0,20,30])
fila_LO['médio'] = fuzz.trapmf(fila_LO.universe, [20,30,60,70])
fila_LO['alto'] = fuzz.trapmf(fila_LO.universe, [60,70,100,100])

# fila_NS
fila_NS['baixo'] = fuzz.trapmf(fila_NS.universe, [0,0,20,30])
fila_NS['médio'] = fuzz.trapmf(fila_NS.universe, [20,30,60,70])
fila_NS['alto'] = fuzz.trapmf(fila_NS.universe, [60,70,100,100])

# tempo_semaforo
tempo_semaforo['muito baixo'] = fuzz.trapmf(tempo_semaforo.universe, [30,30,48,57])
tempo_semaforo['baixo'] = fuzz.trapmf(tempo_semaforo.universe, [48,57,70.5,79.5])
tempo_semaforo['médio'] = fuzz.trapmf(tempo_semaforo.universe, [70.5,79.5,93,102])
tempo_semaforo['alto'] = fuzz.trapmf(tempo_semaforo.universe, [93,102,120,120])

In [6]:
# definir as regras
regra1 = ctrl.Rule(fila_LO['alto'] & (fila_NS['baixo'] | fila_NS['médio']), tempo_semaforo['muito baixo'])
regra2 = ctrl.Rule(fila_LO['alto'] & fila_NS['alto'], tempo_semaforo['baixo'])
regra3 = ctrl.Rule(fila_LO['médio'] & (fila_NS['baixo'] | fila_NS['médio']), tempo_semaforo['baixo'])
regra4 = ctrl.Rule(fila_LO['médio'] & fila_NS['alto'], tempo_semaforo['médio'])
regra5 = ctrl.Rule(fila_LO['baixo'] & (fila_NS['baixo'] | fila_NS['médio']), tempo_semaforo['médio'])
regra6 = ctrl.Rule(fila_LO['baixo'] & fila_NS['alto'], tempo_semaforo['alto'])

In [7]:
# Agregando as regras e defuzificando
semaforo_ctrl = ctrl.ControlSystem([regra1, regra2, regra3, regra4, regra5, regra6])
semaforo_system = ctrl.ControlSystemSimulation(semaforo_ctrl)

In [8]:
def semaphore_time(qtd_c1, qtd_c2):
    semaforo_system.input['fila_LO'] = qtd_c1
    semaforo_system.input['fila_NS'] = qtd_c2
    semaforo_system.compute()
    return semaforo_system.output['tempo_semaforo']

In [3]:
class CarQueue:
    q = 0
    probInc = [0,0,0,1,1,1,2,2]
    probDec = [2,2,2,2,1]
    def increment(self):
        self.q = self.q + rdm.choice(self.probInc)
    def decrement(self):
        if self.q >= 2:
            self.q = self.q - rdm.choice(self.probDec)
        else:
            self.q = 0

In [10]:
c1 = CarQueue()
c2 = CarQueue()

LO_verde = True
counter = 60
while(True):
    print("qtd c1:", c1.q)
    print("qtd c2:", c2.q)
    print("semaphore:", counter, "\n")
    tm.sleep(0.1)
    if LO_verde:
        c1.decrement()
        if (c1.q > 1):
            c1.increment()
        c2.increment()
    else:
        c2.decrement()
        if (c2.q > 1):
            c2.increment()
        c1.increment()
    counter = counter - 1
    if counter <= 0:
        if LO_verde:
            counter = semaphore_time(c1.q, c2.q)
        else:
            counter = semaphore_time(c2.q, c1.q)
        LO_verde = not LO_verde

qtd c1: 0
qtd c2: 0
semaphore: 60 

qtd c1: 0
qtd c2: 0
semaphore: 59 

qtd c1: 0
qtd c2: 1
semaphore: 58 

qtd c1: 0
qtd c2: 1
semaphore: 57 

qtd c1: 0
qtd c2: 1
semaphore: 56 

qtd c1: 0
qtd c2: 3
semaphore: 55 

qtd c1: 0
qtd c2: 4
semaphore: 54 

qtd c1: 0
qtd c2: 5
semaphore: 53 

qtd c1: 0
qtd c2: 6
semaphore: 52 

qtd c1: 0
qtd c2: 6
semaphore: 51 

qtd c1: 0
qtd c2: 8
semaphore: 50 

qtd c1: 0
qtd c2: 8
semaphore: 49 

qtd c1: 0
qtd c2: 10
semaphore: 48 

qtd c1: 0
qtd c2: 11
semaphore: 47 

qtd c1: 0
qtd c2: 12
semaphore: 46 

qtd c1: 0
qtd c2: 13
semaphore: 45 

qtd c1: 0
qtd c2: 13
semaphore: 44 

qtd c1: 0
qtd c2: 15
semaphore: 43 

qtd c1: 0
qtd c2: 15
semaphore: 42 

qtd c1: 0
qtd c2: 16
semaphore: 41 

qtd c1: 0
qtd c2: 16
semaphore: 40 

qtd c1: 0
qtd c2: 18
semaphore: 39 

qtd c1: 0
qtd c2: 19
semaphore: 38 

qtd c1: 0
qtd c2: 19
semaphore: 37 

qtd c1: 0
qtd c2: 19
semaphore: 36 

qtd c1: 0
qtd c2: 20
semaphore: 35 

qtd c1: 0
qtd c2: 20
semaphore: 34 

qtd c1: 0
qtd

KeyboardInterrupt: 